In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from numpy import transpose as t
from numpy.linalg import inv

In [2]:
class LocalLogisticRegression:
    
    def __init__(self,sigma=1):
        self.sigma = sigma
    
    def gaussian_kernel(self,sigma,x_0,x):
        f = 1/np.sqrt(2*np.pi*self.sigma**2)*np.exp(-1/(2*self.sigma**2)*(x-x_0)**2) #mu x_0 sigma=1
        return f
    
    def prob(self,x,beta):
        p = np.exp(x@beta)/(1+np.exp(x@beta))
        return p
        
    def predict(self,x,y,newx):
        x = np.array(x); y = np.array(y)
        n = x.shape[0]
        x = t(np.append([1]*n,np.array(x)).reshape(2,-1)) 
        pred = []
        beta = np.array([0]*2)
        for x0 in newx: 
            beta = np.array([0]*2)
            error = 10000
            while error>0.001:
                l1 = sum([LocalLogisticRegression.gaussian_kernel(self,1,x0,x[i][1])*x[i]*(y[i]-LocalLogisticRegression.prob(self,x[i],beta)) for i in range(n)])
                l2 = sum([-LocalLogisticRegression.gaussian_kernel(self,1,x0,x[i][1])*np.outer(x[i],x[i])*LocalLogisticRegression.prob(self,x[i],beta)*(1-LocalLogisticRegression.prob(self,x[i],beta)) for i in range(n)])
                beta_old = beta
                beta =  beta - inv(l2)@l1
                error = max(abs(beta-beta_old))
            prob = LocalLogisticRegression.prob(self,[1,x0],beta)
            pred.append((prob>0.5).astype('int'))
        return pred

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X[:,0], y, test_size=0.3, random_state=0)

In [4]:
lg = LocalLogisticRegression()

In [5]:
pred = lg.predict(X_train,y_train,X_test)

In [6]:
from sklearn.metrics import zero_one_loss
zero_one_loss(pred, y_test)

0.09941520467836262